In [1]:
import getpass
import pandas as pd
import requests
import json

Bouw de $lookup Bundle adhv input spreadsheet met codes+system

In [122]:
df = pd.read_excel("codelijsten.xls", "Blad3")


In [123]:
payload = {
   "type" : "batch",
   "resourceType" : "Bundle",
   "entry" : [
   ]
}

tolookup = []
done = []
for row in df.iterrows():
    # row[1] is a Panda Series, convert to a list array
    _cells = row[1].to_list()
    _code = _cells[1]
    _system = _cells[2]
    # only do if not already done
    if (not(_code in done)):
        done.append(_code)
        entry = {
             "resource" : {
                "parameter" : [
                   {
                      "name" : "system",
                      "valueUri" : _system
                   },
                   {
                      "name" : "code",
                      "valueCode" : _code
                   }
                ],
                "resourceType" : "Parameters"
             },
             "request" : {
                "method" : "POST",
                "url" : "CodeSystem/$lookup"
             }
          }
        payload["entry"].append(entry)
        tolookup.append ({
            "code": _code,
            "system": _system,
        })
        
#print (json.dumps(payload))
print (json.dumps(tolookup))

[{"code": 258348004, "system": "http://snomed.info/sct"}, {"code": 419511003, "system": "http://snomed.info/sct"}, {"code": 418471000, "system": "http://snomed.info/sct"}, {"code": 426232007, "system": "http://snomed.info/sct"}, {"code": 419199007, "system": "http://snomed.info/sct"}, {"code": 219006, "system": "http://snomed.info/sct"}, {"code": 105542008, "system": "http://snomed.info/sct"}, {"code": 82581004, "system": "http://snomed.info/sct"}, {"code": 228274009, "system": "http://snomed.info/sct"}, {"code": 783261004, "system": "http://snomed.info/sct"}, {"code": 1141703009, "system": "http://snomed.info/sct"}, {"code": 1631000175102, "system": "http://snomed.info/sct"}, {"code": 305351004, "system": "http://snomed.info/sct"}, {"code": 89666000, "system": "http://snomed.info/sct"}, {"code": 40617009, "system": "http://snomed.info/sct"}, {"code": 116762002, "system": "http://snomed.info/sct"}, {"code": 39579001, "system": "http://snomed.info/sct"}, {"code": 126485001, "system": "h

Gebruik nu de NTS om de $lookup Bundle uit te voeren

In [58]:
username=input("NTS username: ")
password = getpass.getpass("NTS password: ")

NTS username: m.van.der.zel@umcg.nl
NTS password: ········


In [106]:
url = "https://terminologieserver.nl/auth/realms/nictiz/protocol/openid-connect/token"

apayload='username='+username+'&password='+password+'&grant_type=password&client_id=cli_client'
headers = { 'Content-Type': 'application/x-www-form-urlencoded' }

response = requests.request("POST", url, headers=headers, data=apayload)

token = response.json()
access_token = token["access_token"]

In [124]:
url = "https://terminologieserver.nl/fhir"

payloadstring = json.dumps(payload)

headers = { 'Authorization': 'Bearer ' + access_token,
          'Content-Type': 'application/fhir+json'}

response = requests.request("POST", url, headers=headers, data=payloadstring)

#print(response.text)

Gebruik response om designations op te halen en zet die in de spreadsheet terug

In [125]:
lookups = response.json()

displays = {}
lookupnr = 0
for lookup in tolookup:
    if (len(lookups["entry"]) > lookupnr):
        if (lookups["entry"][lookupnr]["resource"]["resourceType"] == "Parameters"):
            entryparams = lookups["entry"][lookupnr]["resource"]["parameter"]
            display = [x for x in entryparams if x["name"] == "display"]
            displays[lookup["code"]] = display[0]["valueString"]
    lookupnr = lookupnr + 1

print (displays)

{258348004: 'OTH', 419511003: 'neiging tot ongewenste reactie op medicatie en/of drug', 418471000: 'neiging tot ongewenste reactie op voedsel', 426232007: 'omgevingsgerelateerde allergie', 419199007: 'allergie voor substantie', 219006: 'drinkt alcohol', 105542008: 'drinkt momenteel geen alcohol', 82581004: 'dronk vroeger alcohol', 228274009: 'levenslang niet-drinker', 783261004: 'levenslange niet-drinker van alcohol', 1141703009: 'geen misbruik van alcohol', 1631000175102: 'Patient not asked', 305351004: 'opname op intensive care', 89666000: 'cardiopulmonale resuscitatie', 40617009: 'kunstmatige beademing', 116762002: 'toediening van bloedproduct', 39579001: 'anafylaxie', 126485001: 'urticaria', 267036007: 'dyspneu', 62315008: 'diarree', 418363000: 'jeukende huid', 609433001: 'overgevoeligheid', 16932000: 'misselijkheid en braken', 422587007: 'misselijkheid', 65124004: 'zwelling', 48694002: 'angst', 80313002: 'palpitaties', 40275004: 'contacteczeem', 271807003: 'eruptie van huid', 5119

In [127]:
rownr = 0
for row in df.iterrows():
    #row[1] is a Panda Series, convert to a list array
    _cells = row[1].to_list()
    _code = _cells[1]
    _system = _cells[2]
    if (_code in displays):
        print(_code, ";", displays[_code])
   #     designations = [x for x in entryparams if x["name"] == "designation" and x["part"][0]["valueCode"] == "nl"]
   #     if (designations):
   #         print(rownr, _code, designations[0]["part"][2])
   #     else:
   #         print(rownr, _code, "geen nl designations")
    else:
        print(_code, ";", "? niet gevonden")
    rownr = rownr + 1

258348004 ; OTH
419511003 ; neiging tot ongewenste reactie op medicatie en/of drug
418471000 ; neiging tot ongewenste reactie op voedsel
426232007 ; omgevingsgerelateerde allergie
426232007 ; omgevingsgerelateerde allergie
426232007 ; omgevingsgerelateerde allergie
419199007 ; allergie voor substantie
419511003 ; neiging tot ongewenste reactie op medicatie en/of drug
419511003 ; neiging tot ongewenste reactie op medicatie en/of drug
219006 ; drinkt alcohol
105542008 ; drinkt momenteel geen alcohol
258348004 ; OTH
82581004 ; dronk vroeger alcohol
228274009 ; levenslang niet-drinker
783261004 ; levenslange niet-drinker van alcohol
1141703009 ; geen misbruik van alcohol
1631000175102 ; Patient not asked
305351004 ; opname op intensive care
89666000 ; cardiopulmonale resuscitatie
40617009 ; kunstmatige beademing
116762002 ; toediening van bloedproduct
258348004 ; OTH
39579001 ; anafylaxie
126485001 ; urticaria
267036007 ; dyspneu
62315008 ; diarree
418363000 ; jeukende huid
609433001 ; ove